In [1]:
import rasterio

# Open the DEM file
with rasterio.open("US_merged.tif") as src:
    dem = src.read(1)  # Read the first band (elevation)
    transform = src.transform  # Geospatial transform
    crs = src.crs  # Coordinate reference system

In [2]:
import geopandas as gpd

# Open the GeoJSON file
cellular_data = gpd.read_file("cellular_data.geojson")

In [3]:
# Check CRS of cellular data
print(cellular_data.crs)

# Reproject cellular data to match DEM CRS (if necessary)
if cellular_data.crs != crs:
    cellular_data = cellular_data.to_crs(crs)

EPSG:4326


In [4]:
with rasterio.open("US_merged.tif") as src:
    print("DEM bounds:", src.bounds)
    print("DEM shape:", src.shape)  # (height, width)

DEM bounds: BoundingBox(left=-125.00041666667917, bottom=24.500416666674738, right=-66.50041666669244, top=49.500416666669054)
DEM shape: (29999, 70200)


In [5]:
# Check min/max coordinates of cellular data
print("Cellular data bounds:")
print("Min longitude:", cellular_data.geometry.x.min())
print("Max longitude:", cellular_data.geometry.x.max())
print("Min latitude:", cellular_data.geometry.y.min())
print("Max latitude:", cellular_data.geometry.y.max())

Cellular data bounds:
Min longitude: -176.705472221965
Max longitude: 145.79602777736
Min latitude: -14.3222222217781
Max latitude: 71.3179166667209


In [6]:
from shapely.geometry import box

# Get DEM bounds
with rasterio.open("US_merged.tif") as src:
    dem_bounds = box(*src.bounds)

# Filter cellular data to only include points within DEM bounds
cellular_data = cellular_data[cellular_data.geometry.within(dem_bounds)]

In [7]:
with rasterio.open("US_merged.tif") as src:
    transform = src.transform

# Test the transform with a known point
lon, lat = -122.3321, 47.6062  # Example: Seattle coordinates
row, col = ~transform * (lon, lat)
row, col = int(row), int(col)
print(f"Row: {row}, Col: {col}")

Row: 3201, Col: 2272


In [8]:
def get_elevation(lat, lon, dem, transform):
    # Convert lat/long to row/col in the DEM
    row, col = ~transform * (lon, lat)
    row, col = int(row), int(col)
    
    # Check if row/col is within DEM bounds
    if 0 <= row < dem.shape[0] and 0 <= col < dem.shape[1]:
        return dem[row, col]
    else:
        return None  # Return None for out-of-bounds coordinates

# Add elevation to cellular data
cellular_data['elevation'] = cellular_data.apply(
    lambda row: get_elevation(row.geometry.y, row.geometry.x, dem, transform), axis=1
)

In [9]:
cellular_data = cellular_data.dropna(subset=['elevation'])

In [10]:
mean_elevation = cellular_data['elevation'].mean()
cellular_data['elevation'] = cellular_data['elevation'].fillna(mean_elevation)

In [ ]:
# import numpy as np
# from scipy.ndimage import sobel

# # Calculate slope from DEM
# def calculate_slope(dem, transform):
#     x, y = np.gradient(dem)
#     slope = np.arctan(np.sqrt(x**2 + y**2)) * 180 / np.pi
#     return slope

# # Compute slope
# slope = calculate_slope(dem, transform)

# # Add slope to cellular data
# cellular_data['slope'] = cellular_data.apply(
#     lambda row: slope[int(~transform * (row.geometry.x, row.geometry.y)[0]), int(~transform * (row.geometry.x, row.geometry.y)[1])], axis=1
# )

In [11]:
import numpy as np

def calculate_slope(dem):
    """
    Calculate slope from a DEM using numpy.gradient.
    """
    x, y = np.gradient(dem)
    slope = np.arctan(np.sqrt(x**2 + y**2)) * 180 / np.pi
    return slope

In [12]:
from affine import Affine  # Ensure you have the `affine` library installed

def transform_coordinates(coords, transform):
    """
    Transform coordinates using an affine transformation.
    """
    # Apply the inverse of the transform to get row/col indices
    transformed_coords = ~transform * coords.T
    return transformed_coords[1].astype(int), transformed_coords[0].astype(int)  # rows, cols

In [13]:
# def add_slope_to_cellular_data(cellular_data, dem, transform):
#     """
#     Add slope values to cellular_data using vectorized operations.
#     """
#     # Extract coordinates from cellular_data
#     coords = np.array([(row.geometry.x, row.geometry.y) for row in cellular_data.itertuples()])

#     # Transform coordinates to DEM indices
#     rows, cols = transform_coordinates(coords, transform)

#     # Calculate slope from DEM
#     slope = calculate_slope(dem)

#     # Assign slope values to cellular_data
#     cellular_data['slope'] = slope[rows, cols]

#     return cellular_data

In [ ]:
# # Assuming dem and transform are already defined
# # dem: Your Digital Elevation Model (2D numpy array)
# # transform: Affine transformation for the DEM

# # Add slope to cellular_data
# cellular_data = add_slope_to_cellular_data(cellular_data, dem, transform)

# # Check the result
# print(cellular_data.head())

my kernel keeps dying because of the large dem size, so I'm trying to use chunks.

In [2]:
import geopandas as gpd

In [13]:
import numpy as np
import rasterio
from rasterio.windows import Window
from affine import Affine
import geopandas as gpd

# Open the DEM file
with rasterio.open("US_merged.tif") as src:
    transform = src.transform
    height, width = src.shape

    # Define chunk size
    chunk_size = 5000

    # Extract coordinates from cellular_data
    coords = np.array([(row.geometry.x, row.geometry.y) for row in cellular_data.itertuples()])
    rows, cols = transform_coordinates(coords, transform)

    # Initialize a column for slope values
    cellular_data['slope'] = np.nan

    # Loop through the DEM in chunks
    for i in range(0, height, chunk_size):
        for j in range(0, width, chunk_size):
            window = Window(j, i, min(chunk_size, width - j), min(chunk_size, height - i))
            dem_chunk = src.read(1, window=window)
            slope_chunk = calculate_slope(dem_chunk)

            in_chunk = (
                (rows >= i) & (rows < i + chunk_size) &
                (cols >= j) & (cols < j + chunk_size)
            )
            cellular_data.loc[in_chunk, 'slope'] = slope_chunk[
                rows[in_chunk] - i, cols[in_chunk] - j
            ]

# Save the results
cellular_data.to_file("cellular_data_with_slope.geojson", driver="GeoJSON")

In [14]:
import geopandas as gpd

# Load the cellular data with slope
cellular_data = gpd.read_file("cellular_data_with_slope.geojson")

# Check if the file is empty
print("Cellular Data with Slope:")
print(cellular_data.head())
print("Number of rows:", len(cellular_data))

Cellular Data with Slope:
   FID  UniqSysID                     Licensee Callsign  LocNum  LatDeg  \
0   27      11510  AT&T Mobility Spectrum, LLC  KNKA679      12      47   
1   28      11510  AT&T Mobility Spectrum, LLC  KNKA679       6      47   
2   29      11510  AT&T Mobility Spectrum, LLC  KNKA679       7      47   
3   30      11510  AT&T Mobility Spectrum, LLC  KNKA679       8      47   
4   31      11510  AT&T Mobility Spectrum, LLC  KNKA679       9      47   

   LatMin  LatSec LatDir  LonDeg  ...  SupStruc  AllStruc StrucType LicStatus  \
0      41     1.6      N     122  ...       9.8      12.8         B         A   
1      41    28.0      N     122  ...      45.4      45.4    MTOWER         A   
2      28    49.0      N     122  ...     128.9     128.9    GTOWER         A   
3      41    31.2      N     122  ...      46.0      50.9    MTOWER         A   
4      31     1.5      N     122  ...      46.0      47.8    MTOWER         A   

      latdec      londec            

# Now I will start  processing population density data


In [15]:
# Load the population density GeoTIFF 
with rasterio.open("usa_pd_2020_1km_UNadj.tif") as src:
    pop_density = src.read(1)  # Read the first band
    transform = src.transform  # Geospatial transform
    crs = src.crs  # Coordinate reference system

In [16]:
# Load the world GeoJSON file (pop weighted density)
world = gpd.read_file("PWD_2020_national_100m.geojson")

# Check the column names to identify the country attribute
world.columns

Index(['lon', 'lat', 'year', 'ISO', 'ISO_No', 'Name', 'PWC_Lat', 'PWC_Lon',
       'Pop', 'Density',
       ...
       'PWD_P92', 'PWD_P93', 'PWD_P94', 'PWD_P95', 'PWD_P96', 'PWD_P97',
       'PWD_P98', 'PWD_P99', 'PWD_P100', 'geometry'],
      dtype='object', length=115)

In [17]:
# Filter for the United States using the ISO code
us_data = world[world['ISO'] == 'USA']

# Check the result
print(us_data.head())

# Save the filtered data to a new GeoJSON file
us_data.to_file("us_population.geojson", driver="GeoJSON")

       lon      lat  year  ISO ISO_No           Name  PWC_Lat   PWC_Lon  \
2 -92.4788  37.3172  2020  USA    840  United States  37.3172  -92.4788   

         Pop Density  ...  PWD_P92  PWD_P93  PWD_P94  PWD_P95  PWD_P96  \
2  331002647    36.1  ...  11054.1  12283.2  13841.5  15874.6  18744.1   

   PWD_P97  PWD_P98  PWD_P99  PWD_P100                  geometry  
2  23143.1  30722.0  47778.3  485077.8  POINT (-92.4788 37.3172)  

[1 rows x 115 columns]


In [3]:
# Load the world GeoJSON file (pop weighted density)
world_sub = gpd.read_file("PWD_2020_sub_national_100m.geojson")

# Check the column names to identify the country attribute
world_sub.columns

Index(['lon', 'lat', 'year', 'ISO', 'ISO_No', 'Country_N', 'Adm_N', 'GID_1',
       'HASC', 'PWC_Lat', 'PWC_Lon', 'Pop', 'Density', 'Area', 'PWD_A',
       'PWD_G', 'PWD_M', 'PWD_D1', 'PWD_D2', 'PWD_D3', 'PWD_D4', 'PWD_D5',
       'PWD_D6', 'PWD_D7', 'PWD_D8', 'PWD_D9', 'PWD_D10', 'geometry'],
      dtype='object')

In [6]:
# Filter for the United States using the ISO code
us_sub_data = world_sub[world_sub['ISO'] == 'USA']

# Check the result
print(us_sub_data.head())

# Save the filtered data to a new GeoJSON file
us_sub_data.to_file("us_sub_population.geojson", driver="GeoJSON")

             lon        lat  year  ISO ISO_No      Country_N       Adm_N  \
3471  -86.745407  33.015068  2020  USA    840  United States     Alabama   
3472 -149.075200  61.505200  2020  USA    840  United States      Alaska   
3473 -111.880226  33.363468  2020  USA    840  United States     Arizona   
3474  -92.735825  35.205795  2020  USA    840  United States    Arkansas   
3475 -119.334503  35.463676  2020  USA    840  United States  California   

        GID_1   HASC           PWC_Lat  ...  PWD_D2  PWD_D3  PWD_D4  PWD_D5  \
3471  USA.1_1  US.AL  33.0150680541992  ...    84.6   179.5   341.5   582.0   
3472  USA.2_1  US.AK           61.5052  ...   156.5   340.6   652.2  1214.0   
3473  USA.3_1  US.AZ   33.363468170166  ...  1009.2  1602.5  2175.9  2769.1   
3474  USA.4_1  US.AR  35.2057952880859  ...    66.5   152.6   337.9   639.9   
3475  USA.5_1  US.CA  35.4636764526367  ...  1800.7  2561.5  3287.9  4072.8   

      PWD_D6  PWD_D7  PWD_D8   PWD_D9   PWD_D10                     